## Import modules

In [1]:
import itertools
import numpy as np
import pandas as pd
from PIL import Image

from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3, GlobalAveragePooling2D

/home/vivekb/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Reading data

In [2]:
import numpy as np
train = np.load('cifar_train.npz')
x_train = train['data']
train_labels = train['labels']

test = np.load('cifar_test.npz')
x_test = test['data']
test_labels = test['labels']

print('train_data:', x_train.shape)
print('train_labels:', train_labels.shape)
print('test_data:', x_test.shape)
print('test_labels:', test_labels.shape)

('train_data:', (50000, 32, 32, 3))
('train_labels:', (50000, 1))
('test_data:', (10000, 32, 32, 3))
('test_labels:', (10000, 1))


In [3]:
batch_size = 64
nb_classes = 10

img_rows, img_cols = 32, 32    # input image dimensions
img_channels = 3               # The CIFAR10 images are RGB.

# Convert class vectors to binary class matrices.
from keras.utils import to_categorical
y_train = to_categorical(train_labels, nb_classes)
y_test = to_categorical(test_labels, nb_classes)

print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


## Resize all the images 

In [4]:
def resize(p, size):
    return Image.fromarray(p).resize(size=(size,size))

def resize_all(arr, size):
    t = []
    for i in range(arr.shape[0]):
        t.append(np.array(resize(arr[i], size)))

    return(np.array(t))

In [5]:
x_train = resize_all(x_train, 224)
x_test = resize_all(x_test, 224)

In [6]:
# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

In [7]:
print(x_train.shape)
print(x_test.shape)

(50000, 224, 224, 3)
(10000, 224, 224, 3)


## CNN Architecture

In [8]:
def model_train(optimizer, lr_rate, freez, nb_epoch=2, n_samples=2000):
    tag = 'model_'+optimizer+'_lr_'+str(lr_rate)+'_freeze_'+str(freez)
    trained_model = InceptionV3(include_top=False,weights='imagenet')
    x = trained_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    pred_inception= Dense(10,activation='softmax')(x)
    model = Model(inputs=trained_model.input,outputs=pred_inception)

    #making the layers of vgg16 non-trainable
    for layer in trained_model.layers:
        layer.trainable=freez

    #compiling the model
    if optimizer=='adam':
        optim = Adam(lr=lr_rate)
    elif optimizer=='sgd':
        optim = SGD(lr=lr_rate)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=optim)

    train_datagen = ImageDataGenerator(rescale=None, 
                                       height_shift_range=0.2, 
                                       width_shift_range=0.2,
                                       horizontal_flip=True, 
                                       rotation_range=10)
    validation_datagen = ImageDataGenerator(rescale=None)

    train_generator = train_datagen.flow(x=x_train[:n_samples], y=y_train[:n_samples],batch_size=batch_size)
    validation_generator = validation_datagen.flow(x=x_test, y=y_test,batch_size=batch_size)

    hist = model.fit_generator(train_generator,
                        steps_per_epoch=train_generator.n//batch_size, # '//' in python returns only the quotient
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.n//batch_size, 
                        verbose=0).history
    
    model.save(tag+'.h5')
    np.save(tag+'.npy', hist)
    
    t_acc = np.min(hist['acc'])
    v_acc = np.min(hist['val_acc'])
    return(t_acc, v_acc)

In [9]:
optimizer_list = ['adam', 'sgd']
lr_rate_list = [0.01,0.001]
freez_list = [True, False]

gs_list = list(itertools.product(optimizer_list, lr_rate_list, freez_list))
print(gs_list)

[('adam', 0.01, True), ('adam', 0.01, False), ('adam', 0.001, True), ('adam', 0.001, False), ('sgd', 0.01, True), ('sgd', 0.01, False), ('sgd', 0.001, True), ('sgd', 0.001, False)]


In [10]:
tr_acc = []
val_acc = []
for op, lr, fr in gs_list:
    print('Running combination - Optimizer:{}, Learning Rate:{}, Freeze Weights:{}'.format(op, lr, fr))
    r1, r2 = model_train(op, lr, fr)
    tr_acc.append(r1)
    val_acc.append(r2)

Running combination - Optimizer:adam, Learning Rate:0.01, Freeze Weights:True
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Running combination - Optimizer:adam, Learning Rate:0.01, Freeze Weights:False
Running combination - Optimizer:adam, Learning Rate:0.001, Freeze Weights:True
Running combination - Optimizer:adam, Learning Rate:0.001, Freeze Weights:False
Running combination - Optimizer:sgd, Learning Rate:0.01, Freeze Weights:True
Running combination - Optimizer:sgd, Learning Rate:0.01, Freeze Weights:False
Running combination - Optimizer:sgd, Learning Rate:0.001, Freeze Weights:True
Running combination - Optimizer:sgd, Learning Rate:0.001, Freeze Weights:False


In [11]:
df = pd.DataFrame(gs_list)
df.columns = ['Optimizer', 'Learning Rate', 'Freeze Weights']
df['Train Acc'] = tr_acc
df['Val Acc'] = val_acc

In [12]:
df

,Optimizer,Learning Rate,Freeze Weights,Train Acc,Val Acc
0,adam,0.010,True,0.096074,0.099860
1,adam,0.010,False,0.462198,0.534655
2,adam,0.001,True,0.410282,0.099436
3,adam,0.001,False,0.316532,0.568810
4,sgd,0.010,True,0.444556,0.503305
5,sgd,0.010,False,0.211694,0.362480
6,sgd,0.001,True,0.155242,0.207532
7,sgd,0.001,False,0.121976,0.161959
